In [25]:
# <--- Run this cell to get the Current Yields and TVL

import pandas as pd
import numpy as np
from datetime import date
import json
import joblib
from google.cloud import bigquery
import torch

scaler = joblib.load('X_scaler.joblib')


## TODO: REPLACE WITH IPFS FILE
# Key for a Bigquery Account. The Dataset is Public but you will need a credential to query it:
client = bigquery.Client.from_service_account_json('CREDENTIALS.json')

# Get the APY and TVLs from NOYAs BigQuery Table
table_id = 'anomity.noyaModelResults.result_pytorchLinear_20230707'
query_job = client.query(f"SELECT * FROM {table_id}")
df = query_job.to_dataframe() 

df.tail()

C:\Users\alban\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.1.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,timestamp,apy__Arbitrum_curve__dex_FRAX__USDC,apy__Arbitrum_curve__dex_USDC__USDT,apy__Avalanche_curve__dex_USDC__USDT,apy__Ethereum_curve__dex_BUSD__3CRV,apy__Ethereum_curve__dex_DAI__USDC__USDT,apy__Ethereum_curve__dex_DAI__USDC__USDT__SUSD,apy__Ethereum_curve__dex_DOLA__3CRV,apy__Ethereum_curve__dex_FRAX__3CRV,apy__Ethereum_curve__dex_FRAX__USDC,...,real_apy,date_of_change,current_position_1,current_bestPool_1,max_apy,bridgeFee,gasFee,swapFee,modelScore,NoyaStrategy_cum
507,2022-02-24,NaN,NaN,NaN,0.000046,0.000029,0.000065,NaN,0.000099,NaN,...,0.000138,NaT,1.0,Ethereum_curve__dex_USDN__3CRV,0.000156,0.0,0.000087,0.0002,1.212738e-08,0.003055
508,2022-03-11,NaN,NaN,NaN,0.000064,0.000026,0.000098,NaN,0.000084,NaN,...,0.000122,NaT,1.0,Ethereum_curve__dex_USDN__3CRV,0.000136,0.0,0.000087,0.0002,9.079526e-02,0.006294
509,2022-03-05,NaN,NaN,NaN,0.000057,0.000027,0.000308,NaN,0.000096,NaN,...,0.000193,NaT,1.0,Ethereum_curve__dex_USDN__3CRV,0.000305,0.0,0.000087,0.0004,1.533244e-02,0.005068
510,2022-05-12,NaN,0.000001,NaN,0.000556,0.000259,0.000773,NaN,0.000244,NaN,...,0.000363,NaT,1.0,Ethereum_curve__dex_USDN__3CRV,0.000758,0.0,0.000085,0.0004,7.880231e-07,0.029342
511,2022-07-01,NaN,0.000001,NaN,0.000109,0.000024,0.000290,NaN,0.000075,NaN,...,0.000205,NaT,1.0,Ethereum_curve__dex_USDN__3CRV,0.000288,0.0,0.000082,0.0004,4.028200e-01,0.067290


In [9]:
# <--- Run this cell to Generate the input.json file

# Get the Chain and Token to generate the Dataset
## TODO: REPLACE WITH IPFS FILE FOR POOLS TO VALIDATE
Chain = 'Optimism'
Token = 'FRAX__USDC'
Model_Output = 0.9740270376205444

# Function to Generate the DataFrame for the predictive model
def createDataset(chain, protocol, token):
    col = f'apy__{chain}_{protocol}_{token}'
    for d in [1, 3, 7, 14, 30]:
        # Same Pool Variation
        baseCoin.loc[col,f'apy-Var{d}Day'] = df.loc[df.index[i-d], col] / df.loc[df.index[i], col] - 1
        baseCoin.loc[col,f'tvlUsd-Var{d}Day'] = df.loc[df.index[i-d], f"tvlUsd__{col[5:]}"] / df.loc[df.index[i], f"tvlUsd__{col[5:]}"] - 1

        # Average of the chain/protocol/token Variation
        baseCoin.loc[col,f'apy-chain-mean-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"apy.*{chain}")].mean() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"apy.*{chain}")].mean() - 1
        baseCoin.loc[col,f'tvlUsd-chain-mean-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"tvlUsd.*{chain}")].mean() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"tvlUsd.*{chain}")].mean() - 1
        baseCoin.loc[col,f'apy-protocol-mean-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"apy.*{protocol}")].mean() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"apy.*{protocol}")].mean() - 1
        baseCoin.loc[col,f'tvlUsd-protocol-mean-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"tvlUsd.*{protocol}")].mean() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"tvlUsd.*{protocol}")].mean() - 1
        baseCoin.loc[col,f'apy-token-mean-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"apy.*{token}")].mean() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"apy.*{token}")].mean() - 1
        baseCoin.loc[col,f'tvlUsd-token-mean-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"tvlUsd.*{token}")].mean() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"tvlUsd.*{token}")].mean() - 1

        # Max Yield of the chain/protocol/token Variation
        baseCoin.loc[col,f'apy-chain-max-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"apy.*{chain}")].max() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"apy.*{chain}")].max() - 1
        baseCoin.loc[col,f'apy-protocol-max-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"apy.*{protocol}")].max() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"apy.*{protocol}")].max() - 1
        baseCoin.loc[col,f'apy-token-max-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"apy.*{token}")].max() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"apy.*{token}")].max() - 1

        # Sum of TVL of the chain/protocol/token Variation
        baseCoin.loc[col,f'tvlUsd-chain-sum-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"tvlUsd.*{chain}")].sum() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"tvlUsd.*{chain}")].sum() - 1
        baseCoin.loc[col,f'tvlUsd-protocol-sum-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"tvlUsd.*{protocol}")].sum() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"tvlUsd.*{protocol}")].sum() - 1
        baseCoin.loc[col,f'tvlUsd-token-sum-Var{d}Day'] = df.loc[df.index[i-d], df.columns.str.contains(f"tvlUsd.*{token}")].sum() / \
                                                        df.loc[df.index[i], df.columns.str.contains(f"tvlUsd.*{token}")].sum() - 1

# Set to the last day to calculate the Dataset
i = len(df) - 1 
Protocol =  'curve__dex'

# Generate Dataset
baseCoin = pd.DataFrame()
createDataset(Chain, Protocol, Token)
baseCoin.fillna(0, inplace=True)
baseCoin.replace([np.inf, -np.inf], 0, inplace=True)

input_array = scaler.transform(baseCoin.iloc[-1].values.reshape(1, -1))

x = torch.tensor(input_array)
new_shape = tuple([1]+list(x.shape))
x = torch.reshape(x, new_shape)

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_shapes=[[len(input_array[0])]],
            input_data=[data_array],
            output_data=[[Model_Output]])

with open(f'input-{baseCoin.index[-1][5:]}-{df.index[i].strftime("%Y-%m-%d")}.json', 'w') as file:
 file.write(json.dumps(data))